In [1]:
from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization,\
                                    AveragePooling2D, Flatten, Dense, Dropout,\
                                    Add
from tensorflow.keras.models import Model

/home/ashish/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Number of Parameters to be passed
    i) Array conatining the length of the each dense block
    ii) number of filter array

In [2]:
def create_seizure_net(num_filters_array, num_layer_array):
    inputs = Input(shape=(224, 224, 3))
    conv_pool = Conv2D(64, kernel_size=(7, 7), activation='relu', strides=(2, 2), padding='same')(inputs)
    conv_pool = BatchNormalization()(conv_pool)
    conv_pool = ReLU()(conv_pool)
    conv_pool = AveragePooling2D(pool_size=(2, 2))(conv_pool)
    
    last_layer = Conv2D(num_filters_array[0], kernel_size=(1, 1), padding='same')(conv_pool)
    db = create_dense_block(num_layer_array[0], last_layer, num_filters_array[0])
    conv_layer = create_conv_block(db, int(num_filters_array[0]/2))
    
    last_layer = Conv2D(num_filters_array[1], kernel_size=(1, 1), padding='same')(conv_layer)
    db = create_dense_block(num_layer_array[1], last_layer, num_filters_array[1])
    conv_layer = create_conv_block(db, int(num_filters_array[1]/2))
    
    last_layer = Conv2D(num_filters_array[2], kernel_size=(1, 1), padding='same')(conv_layer)
    db = create_dense_block(num_layer_array[2], last_layer, num_filters_array[2])
    conv_layer = create_conv_block(db, int(num_filters_array[2]/2))
    
    last_layer = Conv2D(num_filters_array[3], kernel_size=(1, 1), padding='same')(conv_layer)
    db = create_dense_block(num_layer_array[3], last_layer, num_filters_array[3])
    db = Conv2D(num_filters_array[3], kernel_size=(7, 7), activation='relu')(db)

    t = Flatten()(db)
    outputs = Dense(5, activation='softmax')(t)
    model = Model(inputs, outputs)
    return model

In [3]:
def create_dense_block(num_layers, last_layer, num_filters):
    dl1 = create_dense_layer(last_layer, num_filters)
    for num in range(num_layers):
        prev_layer = Add()([last_layer, dl1])
        dl2 = create_dense_layer(prev_layer, num_filters)
        last_layer = dl1
        dl1 = dl2
    prev_layer = Add()([last_layer, dl1])
    return prev_layer

In [4]:
def create_dense_layer(prev_layers, num_filters):
    conv_pool = Conv2D(num_filters, kernel_size=(1, 1), activation='relu', strides=1, padding="same")(prev_layers)
    conv_pool = BatchNormalization()(conv_pool)
    conv_pool = ReLU()(conv_pool)
    conv_pool = Conv2D(num_filters, kernel_size=(3, 3), activation='relu', strides=1, padding="same")(conv_pool)
    conv_pool = BatchNormalization()(conv_pool)
    conv_pool = ReLU()(conv_pool)
    conv_pool = Dropout(0.5)(conv_pool)
    return conv_pool
    

In [5]:
def create_conv_block(prev_layers, num_filters):
    mid_layer = Conv2D(num_filters, kernel_size=(1, 1), activation='relu', strides=1, padding='same')(prev_layers)
    mid_layer = BatchNormalization()(mid_layer)
    mid_layer = AveragePooling2D(pool_size=(2, 2))(mid_layer)
    return mid_layer

In [ ]:
num_filters_array = [256, 512, 1280, 1664]
num_layer_array = [5, 11, 35, 23]
model = create_seizure_net(num_filters_array, num_layer_array)
model.summary()